# Importing Dependencies

### Instance Segmentation of Powder Particles and Satellites

This example is used to generate a visualization of an individual image


In [1]:
## regular module imports
import cv2
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import pickle
import skimage.io
import sys

## detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import (
    DatasetCatalog,
    MetadataCatalog,
)
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.structures import BoxMode
#from detectron2.evaluation import coco_evaluation
from detectron2.data.datasets.coco import convert_to_coco_json
from detectron2.evaluation.coco_evaluation import instances_to_coco_json
from detectron2.utils.visualizer import GenericMask
import pycocotools.mask as mask_util
from skimage import measure
from imantics import Polygons, Mask




## Setting System Path

In [2]:
## ampis
root = '../../../'
ocean_images = root + '../../../../ocean/projects/dmr200021p/sprice/initial_paper_complete_set'
sys.path.append(root)
from ampis import data_utils, visualize, export_anno

In [3]:
EXPERIMENT_NAME = 'satellite' # can be 'particles' or 'satellite'
cfg = get_cfg() # initialize cfg object
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))  # load default parameters for Mask R-CNN
cfg.MODEL.DEVICE='cuda'  # 'cpu' to force model to run on cpu, 'cuda' if you have a compatible gpu
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # Since we are training separate models for particles and satellites there is only one class output
cfg.TEST.DETECTIONS_PER_IMAGE = 400 if EXPERIMENT_NAME == 'particle' else 150  # maximum number of instances that can be detected in an image (this is fixed in mask r-cnn)
                        # Increasing this may improve the training results, but will take longer to run (especially without a gpu!) 

## Establishing model Weights

In [4]:
# model weights will be downloaded if they are not present
cfg.OUTPUT_DIR = str(Path(f'{EXPERIMENT_NAME}_output_img_transformation'))
# make the output directory
os.makedirs(Path(cfg.OUTPUT_DIR), exist_ok=True)
model_checkpoints = sorted(Path(root, 'models', 'satellite_output_auto_W3.4').glob('*.pth'))  # paths to weights saved druing training
#cfg.DATASETS.TEST = (dataset_train, dataset_valid)  # predictor requires this field to not be empty
cfg.MODEL.WEIGHTS = str(model_checkpoints[-1])  # use the last model checkpoint saved during training. If you want to see the performance of other checkpoints you can select a different index from model_checkpoints.
predictor = DefaultPredictor(cfg)  # create predictor object

### Establishing Methods


In [5]:
def flip_save_image(name, horizontally, vertically, save=True):
    new_name = name
    img_path = Path('Auto_annotate_images', image_name +'.png')
    img = cv2.imread(str(img_path))
    if horizontally:
        new_name += 'x'
        img = cv2.flip(img, 1)
    if vertically:
        new_name += 'y'
        img = cv2.flip(img, 0)
    new_img_path = Path('Auto_annotate_images', new_name +'.png')
    if save:
        cv2.imwrite(str(new_img_path), img)
    return new_name

def invert_list(input_list, list_range):
    output_list = []
    for i in input_list:
        output_list.append(i)
    for i in range(len(output_list)):
        output_list[i] = list_range - output_list[i]
    return output_list



def invert_shape(input_dict, img_width, img_height, horizontal, vertical):
    if horizontal: 
        input_dict['shape_attributes']['all_points_x'] = invert_list(input_dict['shape_attributes']['all_points_x'], img_width)
    if vertical: 
        input_dict['shape_attributes']['all_points_y'] = invert_list(input_dict['shape_attributes']['all_points_y'], img_height)
    return input_dict


def invert_x_y_regions(input_list, img_width, img_height, horizontal, vertical):
    output_list = []
    for i in input_list:
        output_list.append(invert_shape(i, img_width, img_height, horizontal, vertical))
    return output_list
###TODO: Finish up this method. The name of the image must be changed, including the additional image size
###Then these methods must be created for both horizontal and verticle shifts
###Create an automated program to create all of the neccesary images and test http://www.learningaboutelectronics.com/Articles/How-to-flip-an-image-horizontally-vertically-in-Python-OpenCV.php#:~:text=To%20horizontally%20flip%20an%20image,1%20(for%20horizontal%20flipping).
###Import new docs into VIA and see how they look
def flip_and_save(name, horizontally, vertically, save=True):
    new_name = name
    img_path = Path(root, '..', 'SEM_Images', 'initial_paper_complete_set', name +'.png')
    img = cv2.imread(str(img_path))
    if horizontally:
        new_name += 'X'
        img = cv2.flip(img, 1)
    if vertically:
        new_name += 'Y'
        img = cv2.flip(img, 0)
    new_img_path = Path(root, '..', 'SEM_Images', 'initial_paper_complete_set', 'geometric', new_name +'.png')
    if save:
        cv2.imwrite(str(new_img_path), img)
    return new_name
print('')

def color_and_save(name, transformation):
    #transformation: 0-1 = darker, 1 = no change, 1+ = lighter
    im = Image.open(root + '../SEM_Images/initial_paper_complete_set/geometric/' + name + '.png')
    enhancer = ImageEnhance.Brightness(im)
    factor = transformation
    im_output = enhancer.enhance(factor)
    name_change = name
    if factor < 1:
        name_change += 'd'
    elif factor > 1: 
        name_change += 'b'
    else:
        name_change += 's'
    im_output.save(root + '../SEM_Images/initial_paper_complete_set/photometric/' + name_change + '.png')

    

In [6]:
image_name = "S02_02_SE1_300X18"
img_path = Path(ocean_images, image_name +'.png')
image_size = os.path.getsize(img_path)
print(image_size)
import PIL
image = PIL.Image.open(img_path)
width, height = image.size
print(width, height)

491805
1024 768


### Temporary data annotations for single image

In [39]:
data = {'_via_settings': {'ui': {'annotation_editor_height': 25, 'annotation_editor_fontsize': 0.8, 'leftsidebar_width': 18, 'image_grid': {'img_height': 80, 'rshape_fill': 'none', 'rshape_fill_opacity': 0.3, 'rshape_stroke': 'yellow', 'rshape_stroke_width': 2, 'show_region_shape': True, 'show_image_policy': 'all'}, 'image': {'region_label': '__via_region_id__', 'region_color': '__via_default_region_color__', 'region_label_font': '10px Sans', 'on_image_annotation_editor_placement': 'NEAR_REGION'}}, 'core': {'buffer_size': '18', 'filepath': {}, 'default_filepath': 'AL_5056/PNG_Format/500x/'}, 'project': {'name': 'satellite_auto_v1.0'}}, '_via_img_metadata': {'HP743_4S_500x.png468987': {'filename': 'HP743_4S_500x.png', 'size': 468987, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [182, 181, 179, 178, 177, 173, 173, 174, 174, 175, 175, 178, 190, 191, 191, 192, 192, 191, 191, 186, 184, 183], 'all_points_y': [302, 303, 303, 304, 304, 308, 314, 315, 317, 318, 319, 322, 322, 321, 319, 318, 310, 309, 308, 303, 303, 302]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [775, 774, 773, 772, 771, 770, 770, 768, 768, 771, 780, 781, 781, 782, 782, 783, 783, 782, 782, 781, 780, 779, 778], 'all_points_y': [383, 384, 384, 385, 385, 386, 387, 389, 397, 400, 400, 399, 398, 397, 396, 395, 387, 386, 385, 385, 384, 384, 383]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [461, 460, 459, 458, 457, 456, 456, 454, 454, 453, 453, 452, 452, 453, 453, 459, 472, 473, 474, 475, 475, 476, 476, 475, 475, 474, 473, 469, 467, 466], 'all_points_y': [601, 602, 602, 603, 603, 604, 605, 607, 608, 609, 611, 612, 617, 618, 619, 621, 620, 619, 619, 618, 617, 616, 609, 608, 607, 606, 606, 602, 602, 601]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [277, 274, 273, 272, 272, 273, 273, 274, 274, 276, 276, 277, 285, 286, 286, 287, 287, 283, 282, 281], 'all_points_y': [219, 222, 222, 223, 225, 226, 227, 228, 229, 231, 232, 233, 233, 232, 231, 230, 224, 220, 220, 219]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [922, 919, 919, 922, 929, 931, 931, 928], 'all_points_y': [159, 162, 167, 170, 170, 168, 162, 159]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [581, 580, 577, 576, 575, 570, 570, 569, 569, 570, 570, 571, 571, 575, 576, 578, 585, 590, 590, 592, 592, 595, 595, 594, 594, 593, 593, 592, 591, 590, 589, 586, 585], 'all_points_y': [615, 616, 616, 617, 617, 622, 624, 625, 634, 635, 636, 637, 638, 642, 642, 644, 644, 639, 638, 636, 635, 632, 622, 621, 620, 619, 618, 618, 617, 617, 616, 616, 615]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [478, 477, 476, 475, 475, 476, 476, 478, 479, 480, 485, 488, 488, 489, 489, 487, 486, 485], 'all_points_y': [307, 308, 308, 309, 315, 316, 318, 320, 320, 321, 321, 318, 317, 316, 310, 308, 308, 307]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [137, 136, 135, 132, 132, 131, 131, 132, 132, 134, 135, 136, 144, 145, 146, 147, 147, 148, 148, 147, 147, 143], 'all_points_y': [137, 138, 138, 141, 143, 144, 147, 148, 150, 152, 152, 153, 153, 152, 152, 151, 150, 149, 144, 143, 141, 137]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [183, 182, 181, 180, 179, 177, 177, 178, 178, 179, 189, 192, 192, 189, 188, 187], 'all_points_y': [213, 214, 214, 215, 215, 217, 223, 224, 225, 226, 226, 223, 217, 214, 214, 213]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [206, 205, 201, 200, 196, 194, 193, 192, 192, 191, 191, 190, 190, 191, 191, 192, 192, 193, 193, 194, 194, 197, 209, 210, 211, 217, 217, 218, 218, 219, 219, 218, 218, 214, 212, 211], 'all_points_y': [249, 250, 250, 251, 251, 253, 253, 254, 255, 256, 257, 258, 268, 269, 271, 272, 273, 274, 275, 276, 277, 280, 280, 279, 279, 273, 272, 271, 270, 269, 256, 255, 254, 250, 250, 249]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [478, 477, 476, 475, 475, 474, 474, 476, 487, 488, 488, 487, 487, 486, 486, 485], 'all_points_y': [464, 465, 465, 466, 467, 468, 471, 473, 473, 472, 470, 468, 467, 466, 465, 464]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [519, 518, 516, 515, 514, 513, 512, 510, 510, 509, 509, 510, 510, 512, 512, 515, 516, 517, 525, 526, 527, 531, 531, 532, 532, 533, 533, 532, 532, 531, 531, 526, 524, 523], 'all_points_y': [696, 697, 697, 698, 698, 699, 699, 701, 703, 704, 709, 710, 713, 715, 716, 719, 719, 720, 720, 719, 719, 715, 714, 713, 711, 710, 706, 705, 704, 703, 702, 697, 697, 696]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [478, 477, 476, 473, 473, 474, 474, 477, 478, 479, 484, 485, 485, 486, 486, 487, 487, 488, 488, 489, 489, 488, 488, 487, 486, 485, 484], 'all_points_y': [542, 543, 543, 546, 552, 553, 554, 557, 557, 558, 558, 557, 556, 555, 554, 553, 552, 551, 549, 548, 547, 546, 544, 544, 543, 543, 542]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [751, 750, 749, 746, 746, 747, 755, 757, 757, 756], 'all_points_y': [169, 170, 170, 173, 179, 180, 180, 178, 170, 169]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [692, 691, 690, 688, 688, 687, 687, 688, 688, 690, 700, 701, 701, 702, 702, 701, 701, 700, 700, 699, 699, 698, 698, 697, 696], 'all_points_y': [139, 140, 140, 142, 143, 144, 153, 154, 155, 157, 157, 156, 155, 154, 153, 152, 150, 149, 147, 146, 143, 142, 140, 140, 139]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [413, 412, 410, 408, 407, 406, 406, 405, 405, 404, 404, 405, 405, 406, 406, 408, 408, 409, 423, 423, 424, 424, 423, 423, 422, 422, 420, 420, 419, 419, 418], 'all_points_y': [540, 541, 541, 543, 543, 544, 545, 546, 548, 549, 551, 552, 554, 555, 556, 558, 559, 560, 560, 559, 558, 550, 549, 548, 547, 546, 544, 543, 542, 541, 540]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [772, 771, 770, 768, 768, 769, 769, 770, 772, 779, 780, 780, 779, 779, 778, 777, 776, 775], 'all_points_y': [318, 319, 319, 321, 325, 326, 327, 327, 329, 329, 328, 322, 321, 320, 320, 319, 319, 318]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [948, 947, 946, 944, 944, 945, 945, 947, 947, 948, 953, 954, 954, 955, 955, 956, 956, 953], 'all_points_y': [154, 155, 155, 157, 159, 160, 162, 164, 165, 166, 166, 165, 164, 163, 162, 161, 157, 154]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [801, 800, 799, 797, 796, 795, 795, 794, 794, 795, 796, 797, 803, 804, 805, 806, 806, 804, 803, 802], 'all_points_y': [107, 108, 108, 110, 110, 111, 112, 113, 116, 117, 117, 118, 118, 117, 117, 116, 110, 108, 108, 107]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [360, 358, 358, 359, 363, 363, 362], 'all_points_y': [151, 153, 158, 159, 159, 152, 151]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [819, 818, 816, 815, 814, 811, 811, 810, 810, 809, 809, 810, 810, 816, 817, 818, 822, 823, 827, 828, 831, 832, 834, 835, 836, 839, 839, 840, 840, 841, 841, 840, 840, 839, 839, 838, 838, 834, 831, 830], 'all_points_y': [491, 492, 492, 493, 493, 496, 497, 498, 500, 501, 509, 510, 512, 518, 518, 519, 519, 520, 520, 519, 519, 518, 518, 517, 517, 514, 513, 512, 511, 510, 506, 505, 501, 500, 498, 497, 496, 492, 492, 491]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [440, 439, 439, 440, 440, 444, 449, 451, 451, 449, 448, 447, 446, 444], 'all_points_y': [499, 500, 505, 506, 507, 511, 511, 509, 504, 502, 502, 501, 501, 499]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [620, 619, 618, 618, 617, 617, 614, 614, 613, 612, 610, 610, 609, 609, 611, 617, 618, 620, 622, 623, 625, 626, 628, 628, 629, 629, 627], 'all_points_y': [696, 697, 697, 698, 699, 701, 704, 705, 706, 706, 708, 709, 710, 712, 714, 714, 713, 713, 711, 711, 709, 709, 707, 706, 705, 698, 696]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [446, 445, 444, 444, 443, 443, 444, 444, 445, 445, 449, 455, 457, 457, 458, 458, 457, 457, 455, 453, 452, 450, 449, 447], 'all_points_y': [294, 295, 295, 296, 297, 303, 304, 305, 306, 307, 311, 311, 309, 308, 307, 301, 300, 299, 297, 297, 296, 296, 295, 295]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [462, 461, 460, 455, 455, 454, 454, 453, 453, 454, 454, 455, 455, 456, 456, 460, 461, 462, 471, 472, 473, 475, 476, 477, 477, 479, 479, 480, 480, 481, 481, 480, 480, 479, 479, 478, 478, 477, 475, 474, 473, 472, 471], 'all_points_y': [650, 651, 651, 656, 657, 658, 659, 660, 666, 667, 670, 671, 672, 673, 674, 678, 678, 679, 679, 678, 678, 676, 676, 675, 674, 672, 671, 670, 668, 667, 663, 662, 658, 657, 656, 655, 654, 654, 652, 652, 651, 651, 650]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [757, 756, 754, 749, 749, 750, 750, 752, 753, 762, 763, 766, 768, 768, 767, 767, 766, 766, 764, 763, 762], 'all_points_y': [21, 22, 22, 27, 35, 36, 37, 37, 38, 38, 37, 37, 35, 31, 30, 26, 25, 24, 22, 22, 21]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [803, 801, 801, 800, 800, 801, 801, 802, 803, 807, 808, 808, 806], 'all_points_y': [88, 90, 91, 92, 93, 94, 95, 95, 96, 96, 95, 90, 88]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [823, 820, 820, 819, 819, 820, 820, 821, 821, 825, 826, 827, 828, 829, 833, 834, 835, 836, 840, 841, 842, 845, 844, 844, 841, 840, 835, 834, 827], 'all_points_y': [424, 427, 429, 430, 435, 436, 437, 438, 439, 443, 443, 444, 444, 445, 445, 446, 446, 445, 445, 444, 444, 441, 440, 439, 436, 436, 431, 431, 424]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [732, 731, 731, 730, 730, 731, 731, 733, 736, 737, 737, 735], 'all_points_y': [77, 78, 79, 80, 81, 82, 83, 85, 85, 84, 79, 77]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [937, 931, 931, 930, 930, 936, 939, 939, 938, 938], 'all_points_y': [628, 634, 636, 637, 638, 638, 635, 630, 629, 628]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [935, 934, 933, 932, 932, 933, 933, 935, 935, 936, 936, 938, 938, 940, 940, 945, 945, 946, 946, 947, 947, 958, 958, 959, 959, 958, 958, 957, 957, 954, 954, 953, 953, 951, 951, 948, 947, 946, 945, 944], 'all_points_y': [542, 543, 543, 544, 546, 547, 548, 550, 551, 552, 553, 555, 556, 558, 559, 564, 565, 566, 567, 568, 571, 571, 569, 568, 563, 562, 558, 557, 556, 553, 552, 551, 550, 548, 547, 544, 544, 543, 543, 542]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [49, 48, 48, 47, 48, 48, 49, 54, 55, 55, 54], 'all_points_y': [119, 120, 121, 122, 123, 124, 125, 125, 124, 120, 119]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [880, 880, 879, 879, 883, 883, 884, 884], 'all_points_y': [105, 106, 107, 112, 112, 111, 110, 105]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [812, 811, 809, 806, 806, 807, 809, 810, 812, 813, 814, 815, 832, 832, 833, 833, 832, 832, 829, 827, 826, 823, 822], 'all_points_y': [0, 1, 1, 4, 7, 8, 8, 9, 9, 10, 10, 11, 11, 10, 9, 8, 7, 5, 2, 2, 1, 1, 0]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [546, 545, 545, 546, 547, 548, 547, 547, 548, 548, 546, 546, 544, 543, 541, 540, 539, 539, 538, 538, 539, 540, 539, 539, 540, 540, 541, 541, 542, 543, 544, 544, 545, 547, 548, 552, 552, 553, 553, 555, 555, 556, 556, 558, 563, 565, 567, 568, 569, 572, 572, 573, 573, 572, 572, 571, 571, 569, 569, 566, 565, 563, 562, 561, 560, 559, 552, 548, 547], 'all_points_y': [127, 128, 129, 130, 130, 131, 132, 133, 134, 135, 137, 138, 140, 140, 142, 142, 141, 140, 140, 141, 141, 142, 143, 144, 145, 147, 148, 150, 151, 151, 152, 153, 153, 155, 155, 159, 160, 161, 162, 164, 166, 167, 168, 170, 170, 168, 168, 167, 167, 164, 161, 160, 147, 146, 144, 143, 141, 139, 138, 135, 135, 133, 133, 132, 132, 131, 132, 128, 128]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [778, 777, 777, 776, 776, 775, 775, 776, 776, 777, 777, 782, 783, 782, 782, 781, 781], 'all_points_y': [614, 615, 616, 617, 621, 622, 623, 624, 627, 628, 629, 629, 628, 627, 618, 617, 614]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [539, 537, 537, 536, 536, 537, 538, 546, 549, 550, 550, 549, 549, 548, 546], 'all_points_y': [128, 130, 131, 132, 139, 139, 140, 140, 137, 137, 132, 131, 130, 130, 128]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [664, 663, 662, 660, 660, 661, 661, 664, 667, 669, 669, 670, 670, 669, 669, 668, 667, 666], 'all_points_y': [162, 163, 163, 165, 167, 168, 169, 172, 172, 170, 169, 168, 166, 165, 164, 163, 163, 162]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [335, 334, 333, 333, 334, 336, 337, 337, 336], 'all_points_y': [593, 594, 594, 598, 599, 599, 598, 594, 594]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [885, 884, 884, 886, 893, 894, 894, 893, 892, 891], 'all_points_y': [738, 739, 742, 744, 744, 743, 740, 739, 739, 738]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [396, 393, 393, 394, 394, 397, 398, 399, 400, 400, 401, 401, 402, 402, 401, 401, 400], 'all_points_y': [692, 695, 704, 705, 706, 709, 709, 710, 709, 708, 707, 705, 704, 695, 694, 693, 692]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [305, 302, 302, 308, 309, 309, 310, 310, 309, 309, 307], 'all_points_y': [278, 281, 288, 288, 287, 286, 285, 283, 282, 280, 278]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [690, 683, 682, 680, 680, 681, 682, 683, 684, 686, 687, 690, 696, 698, 701, 702, 706, 709, 709, 708, 708, 706, 705, 704, 703, 702], 'all_points_y': [613, 620, 620, 622, 627, 628, 628, 629, 629, 631, 631, 634, 634, 632, 632, 631, 631, 628, 619, 618, 617, 615, 615, 614, 614, 613]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [901, 900, 903, 904, 905, 906, 907, 910, 911, 911, 910, 910, 908], 'all_points_y': [223, 224, 227, 227, 228, 228, 229, 229, 228, 227, 226, 225, 223]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [498, 497, 496, 495, 495, 493, 492, 490, 490, 491, 491, 498, 510, 511, 512, 515, 515, 514, 513, 512, 510, 509, 508, 507, 505, 504], 'all_points_y': [581, 582, 582, 583, 585, 587, 587, 589, 590, 591, 593, 600, 600, 599, 599, 596, 584, 583, 583, 582, 582, 583, 583, 582, 582, 581]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [497, 495, 495, 494, 494, 496, 501, 503, 503, 502], 'all_points_y': [710, 712, 713, 714, 716, 718, 718, 716, 711, 710]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [880, 878, 878, 879, 883, 883, 882], 'all_points_y': [351, 353, 356, 357, 357, 352, 351]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [237, 236, 235, 233, 233, 232, 232, 233, 233, 242, 243, 243, 244, 244, 243, 243, 242, 242, 241, 240], 'all_points_y': [604, 605, 605, 607, 609, 610, 614, 615, 616, 616, 615, 613, 612, 609, 608, 607, 606, 605, 605, 604]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [873, 871, 871, 872, 877, 877, 875], 'all_points_y': [315, 317, 320, 321, 321, 317, 315]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [903, 902, 901, 900, 900, 902, 903, 904, 904], 'all_points_y': [357, 358, 358, 359, 363, 365, 364, 364, 358]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [6, 5, 5, 7, 12, 12, 9], 'all_points_y': [85, 86, 90, 92, 92, 88, 85]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [472, 471, 471, 470, 470, 471, 476, 476, 475, 474], 'all_points_y': [513, 514, 515, 516, 517, 518, 518, 514, 514, 513]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [425, 424, 424, 423, 423, 424, 424, 426, 427, 428, 431, 431, 429, 429, 427, 427, 426, 426], 'all_points_y': [581, 582, 583, 584, 592, 593, 594, 596, 596, 597, 597, 593, 591, 590, 588, 587, 586, 581]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [888, 887, 886, 886, 887, 893, 893, 892, 891], 'all_points_y': [409, 410, 410, 413, 414, 414, 410, 410, 409]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [204, 203, 203, 202, 202, 205, 207, 208, 210, 211, 211, 210, 210, 209, 207, 206], 'all_points_y': [500, 501, 502, 503, 507, 510, 510, 511, 511, 510, 505, 504, 502, 501, 501, 500]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [916, 914, 910, 909, 908, 905, 904, 904, 905, 905, 906, 906, 908, 908, 909, 909, 910, 910, 921, 921, 922, 923, 925, 926, 926, 925, 925, 924, 924, 925, 925, 924, 924, 921], 'all_points_y': [521, 523, 523, 524, 523, 523, 524, 530, 531, 532, 533, 534, 536, 538, 539, 540, 541, 542, 542, 540, 539, 539, 541, 540, 539, 538, 537, 536, 534, 533, 528, 527, 524, 521]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [677, 676, 675, 674, 674, 675, 678, 679, 679], 'all_points_y': [69, 70, 70, 71, 76, 77, 77, 76, 71]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [505, 504, 503, 503, 502, 502, 503, 510, 511, 511, 510, 510, 509, 508], 'all_points_y': [665, 666, 666, 667, 668, 671, 672, 672, 671, 669, 668, 666, 666, 665]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [216, 214, 214, 221, 221, 220, 220], 'all_points_y': [639, 641, 643, 643, 641, 640, 639]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [550, 550, 549, 549, 550, 555, 555, 554], 'all_points_y': [245, 246, 247, 248, 249, 249, 246, 245]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [812, 811, 810, 809, 808, 806, 806, 808, 809, 828, 829, 829, 828, 827, 826, 823, 822], 'all_points_y': [0, 1, 1, 2, 2, 4, 6, 6, 7, 7, 6, 3, 2, 2, 1, 1, 0]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [922, 919, 919, 921, 925, 925, 926, 926, 927, 927, 925], 'all_points_y': [539, 542, 544, 546, 546, 545, 544, 543, 542, 541, 539]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [503, 502, 501, 500, 500, 501, 504, 505, 505], 'all_points_y': [189, 190, 190, 191, 192, 193, 193, 192, 189]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [21, 19, 21, 22, 27, 34, 36, 36, 32, 27], 'all_points_y': [43, 44, 50, 53, 54, 53, 52, 49, 47, 46]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [272, 271, 267, 261, 256, 255, 256, 258, 260, 263, 266, 270], 'all_points_y': [725, 729, 733, 734, 732, 726, 724, 722, 721, 721, 721, 723]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [284, 284, 285, 289, 294, 296, 298, 299, 297, 293, 290, 288], 'all_points_y': [714, 719, 722, 725, 725, 723, 721, 717, 714, 711, 710, 710]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [364, 362, 364, 366, 370, 377, 382, 386, 386, 385, 380, 372], 'all_points_y': [725, 730, 734, 740, 741, 743, 742, 739, 734, 728, 725, 727]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [368, 368, 372, 378, 380, 382, 384, 382, 380, 374, 370], 'all_points_y': [687, 692, 695, 697, 698, 696, 692, 686, 685, 683, 684]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [217, 216, 217, 218, 221, 223, 227, 229, 229, 226, 222], 'all_points_y': [478, 481, 484, 488, 488, 487, 485, 481, 478, 476, 477]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [233, 233, 235, 239, 241, 242, 241, 237], 'all_points_y': [468, 471, 473, 474, 473, 470, 468, 467]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [241, 243, 245, 247, 249, 249, 245, 241], 'all_points_y': [476, 478, 479, 478, 476, 473, 472, 475]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [205, 209, 211, 214, 215, 221, 223, 222, 219, 210, 207], 'all_points_y': [656, 659, 663, 667, 668, 668, 665, 658, 653, 652, 652]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [218, 220, 225, 230, 233, 233, 232, 230, 227, 223], 'all_points_y': [675, 678, 679, 679, 677, 672, 669, 665, 670, 673]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [255, 254, 254, 256, 258, 260, 260, 260, 258], 'all_points_y': [616, 620, 623, 625, 625, 623, 621, 618, 616]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [325, 325, 325, 328, 332, 334, 330, 327], 'all_points_y': [684, 685, 687, 689, 688, 685, 681, 682]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [348, 346, 348, 351, 354, 356, 356, 353], 'all_points_y': [724, 726, 728, 729, 728, 726, 725, 723]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [439, 434, 432, 431, 430, 432, 436, 439, 438, 439], 'all_points_y': [747, 748, 751, 754, 756, 758, 759, 758, 755, 751]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [457, 457, 454, 452, 450, 449, 450, 454], 'all_points_y': [555, 559, 563, 563, 562, 558, 555, 554]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [484, 488, 494, 499, 499, 498, 488], 'all_points_y': [622, 623, 623, 623, 621, 618, 618]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [493, 495, 497, 499, 504, 508, 509, 509, 507, 506, 490], 'all_points_y': [614, 611, 609, 607, 608, 610, 613, 616, 619, 620, 617]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [495, 498, 500, 499, 494, 491, 489], 'all_points_y': [471, 471, 471, 468, 466, 468, 470]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [409, 407, 406, 410, 416, 419, 424, 429, 428, 425, 418, 413], 'all_points_y': [3, 6, 11, 14, 16, 15, 15, 10, 5, 2, 1, 2]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [406, 406, 409, 412, 416, 418, 416, 412], 'all_points_y': [54, 58, 61, 61, 60, 57, 53, 52]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [460, 459, 459, 461, 464, 469, 471, 472, 470, 468, 465], 'all_points_y': [85, 88, 90, 91, 92, 92, 92, 91, 84, 82, 82]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [412, 413, 416, 419, 420, 420, 417, 416, 413], 'all_points_y': [64, 68, 69, 69, 66, 64, 62, 61, 62]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [270, 271, 269, 263, 263, 265, 266], 'all_points_y': [170, 174, 179, 177, 174, 169, 169]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [224, 224, 227, 231, 234, 237, 236, 233, 229, 227], 'all_points_y': [488, 491, 493, 494, 493, 489, 485, 482, 482, 484]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [366, 367, 369, 374, 378, 386, 383, 380, 374], 'all_points_y': [703, 707, 712, 716, 717, 711, 702, 699, 698]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [380, 384, 387, 390, 391, 390, 387, 385, 384, 385, 386, 382, 380, 377], 'all_points_y': [722, 722, 721, 719, 713, 710, 706, 704, 704, 707, 709, 714, 715, 719]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [488, 488, 489, 491, 493, 495, 495, 495, 493, 491], 'all_points_y': [717, 719, 721, 723, 723, 721, 719, 718, 716, 715]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [558, 556, 557, 559, 561, 564, 565, 564], 'all_points_y': [674, 677, 682, 684, 687, 687, 682, 678]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [529, 526, 526, 530, 535, 536, 534, 531], 'all_points_y': [661, 663, 665, 666, 667, 665, 663, 661]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [474, 474, 478, 483, 484, 482, 480], 'all_points_y': [680, 683, 686, 685, 682, 678, 677]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [807, 807, 803, 799, 797, 798, 800, 804, 807, 811, 811], 'all_points_y': [520, 520, 519, 521, 524, 529, 532, 534, 532, 525, 522]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [872, 875, 880, 881, 880, 877, 875], 'all_points_y': [516, 518, 520, 519, 516, 514, 515]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [815, 819, 820, 819, 812, 812, 814], 'all_points_y': [400, 405, 410, 412, 410, 406, 404]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [712, 715, 720, 720, 718, 713, 711, 711], 'all_points_y': [168, 171, 180, 181, 182, 180, 176, 171]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [724, 730, 737, 744, 748, 751, 753, 743, 738, 731, 727, 724], 'all_points_y': [187, 184, 184, 188, 193, 199, 203, 210, 211, 210, 208, 201]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [758, 756, 757, 760, 763, 765, 766, 765, 764], 'all_points_y': [182, 185, 188, 190, 189, 189, 186, 184, 182]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [537, 538, 541, 546, 550, 553, 544, 542, 541, 538], 'all_points_y': [304, 308, 310, 311, 310, 306, 304, 304, 304, 307]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [543, 539, 539, 540, 545, 552, 556, 560, 560, 558, 556], 'all_points_y': [292, 295, 300, 302, 304, 304, 303, 300, 296, 291, 289]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [592, 596, 599, 602, 606, 610, 613, 611, 604, 599], 'all_points_y': [343, 347, 353, 360, 369, 365, 358, 349, 344, 342]}, 'region_attributes': {}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [556, 554, 554, 555, 558, 564, 571, 573, 577, 581, 575, 572, 567, 571, 569, 562], 'all_points_y': [628, 634, 642, 650, 655, 660, 662, 662, 652, 644, 643, 638, 626, 621, 619, 621]}, 'region_attributes': {}}], 'file_attributes': {}}}}

### Loading in annotations

In [13]:
json_path_train = Path('..', 'training_sets', f'{EXPERIMENT_NAME}_auto_training_v1.6.json')  # path to training data
json_path_val = Path('..','training_sets',f'{EXPERIMENT_NAME}_auto_validation_v1.1.json')  # path to training data
assert json_path_train.is_file(), 'training file not found!'
assert json_path_val.is_file(), 'validation file not found!'
f = open(json_path_train)
data = json.load(f)

# Transforming Annotations for Photometric and Geometric Transformations


In [12]:
new_annos = []
new_dict = {}
for i in data['_via_img_metadata']:
    image_names = []
    image_sizes = []
    img_name = i.split('.')[0]
    image_names.append(img_name+'s')
    image_names.append(img_name+'d')
    image_names.append(img_name+'b')
    image_names.append(img_name+'Xb')
    image_names.append(img_name+'Xd')
    image_names.append(img_name+'Xs')
    image_names.append(img_name+'Yb')
    image_names.append(img_name+'Yd')
    image_names.append(img_name+'Ys')
    image_names.append(img_name+'XYs')
    image_names.append(img_name+'XYb')
    image_names.append(img_name+'XYd')
    for j in image_names:
        image_sizes.append(os.path.getsize(Path(ocean_images, 'photometric', j +'.png')))
    #print(data['_via_img_metadata'][i])
    writable_dict = {'regions': data['_via_img_metadata'][i]['regions']}
    #print(writable_dict)
    with open(ocean_images + '/temp_dict1.json', 'w') as t:
        json.dump(writable_dict, t)
    with open(ocean_images + '/temp_dict2.json', 'w') as t:
        json.dump(writable_dict, t)
    with open(ocean_images + '/temp_dict3.json', 'w') as t:
        json.dump(writable_dict, t)
    with open(ocean_images + '/temp_dict4.json', 'w') as t:
        json.dump(writable_dict, t)
    json_temp_path1 = Path(ocean_images, 'temp_dict1.json')
    json_temp1 = open(json_temp_path1)
    initial = json.load(json_temp1)
    json_temp_path2 = Path(ocean_images, 'temp_dict2.json')
    json_temp2 = open(json_temp_path2)
    inverted_x = json.load(json_temp2)
    json_temp_path3 = Path(ocean_images, 'temp_dict3.json')
    json_temp3 = open(json_temp_path3)
    inverted_y = json.load(json_temp3)
    json_temp_path4 = Path(ocean_images, 'temp_dict4.json')
    json_temp4 = open(json_temp_path4)
    inverted_xy = json.load(json_temp4)
    inverted_x['regions'] = invert_x_y_regions(inverted_x['regions'], 1024, 768, False, True)
    inverted_y['regions'] = invert_x_y_regions(inverted_y['regions'], 1024, 768, True, False)
    inverted_xy['regions'] = invert_x_y_regions(inverted_xy['regions'], 1024, 768, True, True)
    print('-'*30)
    for k in range(len(image_names)):
        temp_dict = {}
        temp_dict['filename'] = image_names[k] + '.png'
        temp_dict['size'] = image_sizes[k]
        if k < 3:
            temp_dict['regions'] = initial['regions']
        elif k < 6:
            temp_dict['regions'] = inverted_y['regions']
        elif k < 9:
            temp_dict['regions'] = inverted_x['regions']
        elif k < 12:
            temp_dict['regions'] = inverted_xy['regions']
        new_dict[image_names[k] +'.png' + str(image_sizes[k])] = temp_dict



------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------


### Saving Annotations

In [68]:
with open(ocean_images + '/satellite_auto_training_v2.6.json', 'w') as f:
    json.dump(new_dict, f)
#print("Number of Images", str(len(new_annos)))
#print(new_dict)

# Transforming Annotations for Geometric Transformations


In [10]:
new_annos = []
new_dict = {}
for i in data['_via_img_metadata']:
    image_names = []
    image_sizes = []
    img_name = i.split('.')[0]
    image_names.append(img_name)
    image_names.append(img_name+'X')
    image_names.append(img_name+'Y')
    image_names.append(img_name+'XY')
    for j in image_names:
        image_sizes.append(os.path.getsize(Path(ocean_images, 'geometric', j +'.png')))
    #print(data['_via_img_metadata'][i])
    writable_dict = {'regions': data['_via_img_metadata'][i]['regions']}
    #print(writable_dict)
    with open(ocean_images + '/temp_dict1.json', 'w') as t:
        json.dump(writable_dict, t)
    with open(ocean_images + '/temp_dict2.json', 'w') as t:
        json.dump(writable_dict, t)
    with open(ocean_images + '/temp_dict3.json', 'w') as t:
        json.dump(writable_dict, t)
    with open(ocean_images + '/temp_dict4.json', 'w') as t:
        json.dump(writable_dict, t)
    json_temp_path1 = Path(ocean_images, 'temp_dict1.json')
    json_temp1 = open(json_temp_path1)
    initial = json.load(json_temp1)
    json_temp_path2 = Path(ocean_images, 'temp_dict2.json')
    json_temp2 = open(json_temp_path2)
    inverted_x = json.load(json_temp2)
    json_temp_path3 = Path(ocean_images, 'temp_dict3.json')
    json_temp3 = open(json_temp_path3)
    inverted_y = json.load(json_temp3)
    json_temp_path4 = Path(ocean_images, 'temp_dict4.json')
    json_temp4 = open(json_temp_path4)
    inverted_xy = json.load(json_temp4)
    inverted_x['regions'] = invert_x_y_regions(inverted_x['regions'], 1024, 768, False, True)
    inverted_y['regions'] = invert_x_y_regions(inverted_y['regions'], 1024, 768, True, False)
    inverted_xy['regions'] = invert_x_y_regions(inverted_xy['regions'], 1024, 768, True, True)
    print('-'*30)
    for k in range(len(image_names)):
        temp_dict = {}
        temp_dict['filename'] = image_names[k] + '.png'
        temp_dict['size'] = image_sizes[k]
        if k == 0:
            temp_dict['regions'] = initial['regions']
        elif k == 1:
            temp_dict['regions'] = inverted_y['regions']
        elif k == 2:
            temp_dict['regions'] = inverted_x['regions']
        elif k == 3:
            temp_dict['regions'] = inverted_xy['regions']
        new_dict[image_names[k] +'.png' + str(image_sizes[k])] = temp_dict



------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------


### Saving Annotations

In [9]:
with open(ocean_images + '/satellite_auto_training_v3.6.json', 'w') as f:
    json.dump(new_dict, f)
#print("Number of Images", str(len(new_annos)))
#print(new_dict)

# Transforming Annotations for Photometric Transformations


In [14]:
new_annos = []
new_dict = {}
for i in data['_via_img_metadata']:
    image_names = []
    image_sizes = []
    img_name = i.split('.')[0]
    image_names.append(img_name+'s')
    image_names.append(img_name+'b')
    image_names.append(img_name+'d')
    for j in image_names:
        image_sizes.append(os.path.getsize(Path(ocean_images, 'photometric', j +'.png')))
    #print(data['_via_img_metadata'][i])
    print('-'*30)
    for k in range(len(image_names)):
        temp_dict = {}
        temp_dict['filename'] = image_names[k] + '.png'
        temp_dict['size'] = image_sizes[k]
        temp_dict['regions'] = data['_via_img_metadata'][i]['regions']
        new_dict[image_names[k] +'.png' + str(image_sizes[k])] = temp_dict



------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------


### Saving Annotations

In [15]:
with open(ocean_images + '/satellite_auto_training_v4.6.json', 'w') as f:
    json.dump(new_dict, f)
#print("Number of Images", str(len(new_annos)))
#print(new_dict)